In [1]:
from pyimagesearch import imutils
import numpy as np
import cv2

In [2]:
# define the upper and lower boundaries of the HSV pixel
# intensities to be considered 'skin'
lower = np.array([0,48,80],dtype = "uint8")
upper = np.array([20, 255, 255], dtype = "uint8")

In [3]:
def nothing(x):
    pass

cv2.namedWindow('image')

cv2.createTrackbar('l1','image' ,lower[0],255,nothing)
cv2.createTrackbar('l2','image' ,lower[1],255,nothing)
cv2.createTrackbar('l3','image', lower[2],255,nothing)
cv2.createTrackbar('h1','image',upper[0],255,nothing)
cv2.createTrackbar('h2','image',upper[1],255,nothing)
cv2.createTrackbar('h3','image',upper[2],255,nothing)

camera = cv2.VideoCapture(0)

while True:
    #grab the current fram
    (grabbed, frame) = camera.read()
    
    if not grabbed:
        print(grabbed)
        break
                
    lower[0] = cv2.getTrackbarPos('l1','image') 
    lower[1] = cv2.getTrackbarPos('l2','image')
    lower[2] = cv2.getTrackbarPos('l3','image')
    upper[0] = cv2.getTrackbarPos('h1','image')
    upper[1] = cv2.getTrackbarPos('h2','image')
    upper[2] = cv2.getTrackbarPos('h3','image')
    
    # resize the frame, convert it to the HSV color space,
    # and determine the HSV pixel intensities that fall into
    # the specified upper and lower boundaries
    frame = imutils.resize(frame, width = 400)
    converted = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    skinMask = cv2.inRange(converted, lower, upper)
    
    # apply a series of erosions and dilations to the mask
    # using an elliptical kernel
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11,11))
    skinMask = cv2.erode(skinMask, kernel, iterations = 2)
    skinMask = cv2.dilate(skinMask, kernel, iterations = 2)
    
    # blur the mask to help remove noise, then apply
    # the mask to the frame
    skinMask = cv2.GaussianBlur(skinMask,(3,3), 0)
    skin = cv2.bitwise_and(frame, frame, mask = skinMask)
    
    # show the skin in the image along with the mask
    cv2.imshow("images", np.hstack([frame,skin]))
    
    # if the 'q' key is pressed, stop the loop
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
        
# cleanup the camera and close any open windows
camera.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 